In [1]:
# importing libraries
import pandas as pd
from sklearn import datasets
import tensorflow as tf
import itertools

In [3]:
# import dataset from sklearn
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
          "dis", "tax", "ptratio", "medv"]

In [4]:
# import data and named it columns
training_set = pd.read_csv("boston_train.csv", skipinitialspace=True,skiprows=1, names=COLUMNS)

test_set = pd.read_csv("boston_test.csv", skipinitialspace=True,skiprows=1, names=COLUMNS)

prediction_set = pd.read_csv("boston_predict.csv", skipinitialspace=True,skiprows=1, names=COLUMNS)

In [5]:
# checkout the shape
print(training_set.shape, test_set.shape, prediction_set.shape)

(400, 10) (100, 10) (6, 10)


In [6]:
# labelling features and label columns
FEATURES = ["crim", "zn", "indus", "nox", "rm",
                 "age", "dis", "tax", "ptratio"]
LABEL = "medv"

In [7]:
# initiate feature columns
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

In [8]:
# define optimizer and estimator
my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0000001)

my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

estimator = tf.estimator.LinearRegressor(    
        feature_columns=feature_cols,   
        model_dir="train", optimizer=my_optimizer)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000C0EC306CF8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evalu

In [9]:
# defining input function class
def get_input_fn(data_set, num_epochs=None, n_batch = 128, shuffle=True):    
         return tf.estimator.inputs.pandas_input_fn(       
         x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),       
         y = pd.Series(data_set[LABEL].values),       
         batch_size=n_batch,          
         num_epochs=num_epochs,       
         shuffle=shuffle)

In [10]:
# train the model
estimator.train(input_fn=get_input_fn(training_set,                                       
                                           num_epochs=None,                                      
                                           n_batch = 128,                                      
                                           shuffle=False),                                      
                                           steps=1000)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from train\model.ckpt-1000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 1000 into train\model.ckpt.
INFO:tensorflow:loss = 82497.04, step = 1001
INFO:tensorflow:global_step/sec: 65.6309
INFO:tensorflow:loss

In [11]:
# evaluate the model wit test dataset
ev = estimator.evaluate(    
          input_fn=get_input_fn(test_set,                          
          num_epochs=1,                          
          n_batch = 128,                          
          shuffle=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-14T18:35:19Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-14-18:35:21
INFO:tensorflow:Saving dict for global step 2000: average_loss = 527.42584, global_step = 2000, label/mean = 22.08, loss = 52742.586, prediction/mean = 0.40144965
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: train\model.ckpt-2000


In [12]:
# print the loss value
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

Loss: 52742.585938


In [13]:
training_set['medv'].describe()

count    400.000000
mean      22.625500
std        9.572593
min        5.000000
25%       16.600000
50%       21.400000
75%       25.025000
max       50.000000
Name: medv, dtype: float64

In [14]:
# prediction function
y = estimator.predict(    
         input_fn=get_input_fn(prediction_set,                          
         num_epochs=1,                          
         n_batch = 128,                          
         shuffle=False))

In [15]:
# to print the estimated values
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predictions: [array([0.25447795], dtype=float32), array([0.6718134], dtype=float32), array([0.2887641], dtype=float32), array([0.22756661], dtype=float32), array([0.6730065], dtype=float32), array([0.31221557], dtype=float32)]
